# Préparation des Données pour la Prédiction des Incendies au Portugal / Greece

"""
## Contexte du Projet
Ce projet vise à développer un système de prédiction des incendies, exploitant des informations sur la végétation et les conditions météorologiques. Nous concentrons notre étude sur le Portugal et la Grèce, deux des pays les plus susceptibles à ces catastrophes naturelles en raison de leur climat méditerranéen et de leur végétation dense.

## Source des Données
Les données historiques utilisées ici proviennent d'un dataset mis à disposition par la NASA, qui est accessible publiquement pour des analyses de ce type. Vous pouvez consulter et télécharger le dataset directement depuis https://firms.modaps.eosdis.nasa.gov/country/

## Objectif de cette Partie
L'objectif de ce notebook est de créer un dataframe initial contenant les données historiques d'incendies. Nous structurons ce dataframe avec les colonnes essentielles suivantes :
- 'longitude' : Longitude du lieu de l'incendie.
- 'latitude' : Latitude du lieu de l'incendie.
- 'date' : Date de l'observation de l'incendie.
- 'class' : Catégorie indiquant la présence ou l'absence d'un incendie (fire/not fire).

Ces informations constituent la base de notre analyse et de nos modèles prédictifs futurs.
"""


In [2]:
from geopy.geocoders import Nominatim
import requests
from datetime import datetime
import openmeteo_requests


import pandas as pd
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import requests_cache
import missingno as msno 
import matplotlib.pyplot as plt
from datetime import timedelta
import pandas as pd
import numpy as np
from openmeteo_requests import Client
import requests_cache
from retry_requests import retry


In [28]:
df23 =pd.read_csv("modis_2023_Portugal.csv")

In [29]:
df23.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,38.3511,-8.6941,302.0,1.1,1.1,2023-01-02,2209,Terra,MODIS,46,61.03,277.6,10.3,N,0
1,41.2716,-7.9209,301.4,1.4,1.2,2023-01-05,1130,Terra,MODIS,46,61.03,281.0,11.4,D,0
2,37.9808,-8.8225,310.9,1.1,1.0,2023-01-12,2228,Terra,MODIS,81,61.03,280.8,15.0,N,0
3,37.9822,-8.8107,304.3,1.1,1.0,2023-01-12,2228,Terra,MODIS,60,61.03,280.0,10.5,N,0
4,37.8934,-7.5275,303.9,1.0,1.0,2023-01-20,1348,Aqua,MODIS,58,61.03,288.4,5.9,D,0


In [30]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1062 entries, 0 to 1061
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    1062 non-null   float64
 1   longitude   1062 non-null   float64
 2   brightness  1062 non-null   float64
 3   scan        1062 non-null   float64
 4   track       1062 non-null   float64
 5   acq_date    1062 non-null   object 
 6   acq_time    1062 non-null   int64  
 7   satellite   1062 non-null   object 
 8   instrument  1062 non-null   object 
 9   confidence  1062 non-null   int64  
 10  version     1062 non-null   float64
 11  bright_t31  1062 non-null   float64
 12  frp         1062 non-null   float64
 13  daynight    1062 non-null   object 
 14  type        1062 non-null   int64  
dtypes: float64(8), int64(3), object(4)
memory usage: 124.6+ KB


# Sélection des Données pour l'Analyse des Incendies

"""
## Critères de Sélection des Données
Pour cette analyse, nous nous concentrons uniquement sur les observations classées comme 'type 0' dans les données historiques. Selon les métadonnées fournies avec le dataset, 'type 0' désigne les incendies de forêt ou les feux de végétation, ce qui est central à notre étude sur les incendies au Portugal. 

En outre, afin d'assurer la fiabilité de notre modèle prédictif, nous sélectionnons des observations ayant un niveau de confiance ('confidence') supérieur à 80. Cette mesure de confiance, calculée à partir de l'algorithme du satellite, indique la probabilité que l'observation rapportée soit effectivement un incendie.

## Colonnes Utilisées
Les colonnes utilisées dans notre analyse comprennent :
- 'latitude' et 'longitude' : Coordonnées géographiques de l'incendie.
- 'acq_date' et 'acq_time' : Date et heure de l'acquisition des données.
- 'confidence', 'bright_t31', 'frp' : D'autres indicateurs de la force et de la température du feu.


Ces critères et colonnes garantissent que nous utilisons des données précises et pertinentes pour prédire les incendies de manière efficace.
"""


In [31]:
df23 = df23 [df23["type"] == 0]

In [32]:
df23 = df23[df23 ["confidence"]>= 80]

In [33]:
df23['acq_date'] = pd.to_datetime(df23['acq_date'])

In [34]:
df23['mois'] = df23['acq_date'].dt.to_period('M')

In [35]:
occurrence_par_mois = df23['mois'].value_counts().sort_index().reset_index()
occurrence_par_mois.columns = ['mois', 'Occurrences']

# Affichage des résultats
print(occurrence_par_mois)

       mois  Occurrences
0   2023-01            3
1   2023-02           63
2   2023-03            9
3   2023-04           14
4   2023-05            2
5   2023-07           20
6   2023-08          298
7   2023-09            5
8   2023-10           34
9   2023-11            2
10  2023-12            2


In [36]:
df23.drop(["mois"], axis=1, inplace = True)

**Nous sélectionnons les données correspondant uniquement aux mois de février (02) et août (08). Ces mois ont été identifiés comme ayant un taux historiquement élevé d'incendies, ce qui en fait des périodes critiques pour notre étude de prédiction.**

In [37]:
df23 = df23[df23['acq_date'].dt.month.isin([2, 8])]

In [39]:
df23.drop(["brightness","scan","track","satellite","instrument","confidence","version","bright_t31","frp","daynight","type"], axis =1 , inplace = True)

In [50]:
df23

,latitude,longitude,acq_date,acq_time,class
18,38.0083,-8.1173,2023-02-01,1406,fire
19,40.9942,-7.9938,2023-02-01,1407,fire
22,40.9996,-8.0012,2023-02-01,1407,fire
23,41.6917,-8.4728,2023-02-01,1407,fire
33,40.9666,-7.9809,2023-02-03,1145,fire
...,...,...,...,...,...
913,37.3134,-8.5280,2023-08-28,1420,fire
914,37.3047,-8.5084,2023-08-28,1420,fire
916,37.3151,-8.5116,2023-08-28,1420,fire
919,41.7812,-8.3617,2023-08-28,2235,fire


# Génération de Données 'Not_Fire'

"""
## Méthodologie pour Créer des Données 'Not_Fire'
Afin de construire un ensemble de données équilibré pour notre modèle prédictif d'incendies, nous générons des observations 'not_fire' en utilisant les mêmes coordonnées géographiques que les incendies enregistrés ('fire'). Pour chaque incendie documenté à une date 'x', nous insérons des observations 'not_fire' aux mêmes longitude et latitude, mais à des dates décalées de -10, -20, +10, et +20 jours par rapport à 'x'. 

## Assurer l'Intégrité des Données
Pour garantir l'intégrité des données, nous vérifions que ces dates décalées ne coïncident pas avec d'autres jours où des incendies ont été enregistrés. Cette méthode assure que les données 'not_fire' représentent effectivement des périodes sans incendie.

"""

In [31]:
def generate_not_fire_data(fire_data):
    # Assurez-vous que la colonne des dates est au format datetime
    fire_data['acq_date'] = pd.to_datetime(fire_data['acq_date'])

    # Définir les décalages de temps
    time_offsets = [10, 20, -10, -20]

    # Préparer une liste pour collecter toutes les nouvelles lignes
    new_rows = []

    # Itérer sur chaque ligne dans le DataFrame original
    for index, row in fire_data.iterrows():
        for offset in time_offsets:
            # Créer une nouvelle ligne pour chaque décalage de temps
            new_row = row.copy()
            new_row['acq_date'] = row['acq_date'] + timedelta(days=offset)
            new_rows.append(new_row)

    # Concaténer toutes les nouvelles lignes dans un DataFrame
    not_fire_data = pd.concat([pd.DataFrame([row]) for row in new_rows], ignore_index=True)

    return not_fire_data


In [41]:
not_fire_data =  generate_not_fire_data(df23)

In [42]:
not_fire_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444 entries, 0 to 1443
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   latitude   1444 non-null   float64       
 1   longitude  1444 non-null   float64       
 2   acq_date   1444 non-null   datetime64[ns]
 3   acq_time   1444 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(1)
memory usage: 45.2 KB


In [44]:
not_fire_data = not_fire_data [not_fire_data ['acq_date'].dt.month.isin([2, 8])]

In [51]:
not_fire_data

,latitude,longitude,acq_date,acq_time,class
0,38.0083,-8.1173,2023-02-11,1406,not fire
1,38.0083,-8.1173,2023-02-21,1406,not fire
4,40.9942,-7.9938,2023-02-11,1407,not fire
5,40.9942,-7.9938,2023-02-21,1407,not fire
8,40.9996,-8.0012,2023-02-11,1407,not fire
...,...,...,...,...,...
1435,37.3151,-8.5116,2023-08-08,1420,not fire
1438,41.7812,-8.3617,2023-08-18,2235,not fire
1439,41.7812,-8.3617,2023-08-08,2235,not fire
1442,41.6946,-7.7698,2023-08-20,2218,not fire


#  Exportation des Données 

"""
## Création des Étiquettes et Fusion des Données
Dans cette étape finale, nous attribuons des étiquettes 'fire' aux données historiques d'incendies et 'not_fire' aux données générées précédemment. Ces étiquettes sont essentielles pour le processus d'apprentissage supervisé de notre modèle prédictif.

## Concaténation et Mélange des Données
Une fois les étiquettes assignées, nous concaténons les deux ensembles de données (fire et not_fire) pour former un dataset unifié. Pour éviter tout biais lors de la phase d'apprentissage, ce dataset combiné est mélangé aléatoirement. Ce mélange assure une distribution homogène des différentes classes au sein de l'ensemble d'entraînement.

## Exportation en Fichier CSV
La dernière étape consiste à exporter le dataset finalisé sous forme de fichier .csv. Ce fichier sera utilisé pour les étapes ultérieures de modélisation et de validation du modèle. Le stockage en format CSV facilite également la réutilisation et le partage des données préparées pour des analyses futures ou des validations indépendantes.

Ce processus complet garantit que nous disposons d'un ensemble de données bien préparé, crucial pour le succès de nos prédictions d'incendies.
"""


In [46]:
not_fire_data["class"] = "not fire"
df23["class"] = "fire"

In [47]:
concatenated_df = pd.concat([df23, not_fire_data], ignore_index=True)

In [48]:
shuffled_df = concatenated_df.sample(frac=1).reset_index(drop=True)

In [49]:
shuffled_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1076 entries, 0 to 1075
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   latitude   1076 non-null   float64       
 1   longitude  1076 non-null   float64       
 2   acq_date   1076 non-null   datetime64[ns]
 3   acq_time   1076 non-null   int64         
 4   class      1076 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 42.2+ KB


In [52]:
shuffled_df.to_csv('data_Portugal_.csv', index=False)

## Suivi de la Même Démarche pour les Données de la Grèce
"""
Dans cette section, nous appliquons la même démarche de préparation des données d'incendies que celle utilisée pour le Portugal aux données concernant la Grèce.
"""


In [20]:
df23 =pd.read_csv("modis_2023_Greece.csv")

In [21]:
df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3262 entries, 0 to 3261
Data columns (total 15 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    3262 non-null   float64
 1   longitude   3262 non-null   float64
 2   brightness  3262 non-null   float64
 3   scan        3262 non-null   float64
 4   track       3262 non-null   float64
 5   acq_date    3262 non-null   object 
 6   acq_time    3262 non-null   int64  
 7   satellite   3262 non-null   object 
 8   instrument  3262 non-null   object 
 9   confidence  3262 non-null   int64  
 10  version     3262 non-null   float64
 11  bright_t31  3262 non-null   float64
 12  frp         3262 non-null   float64
 13  daynight    3262 non-null   object 
 14  type        3262 non-null   int64  
dtypes: float64(8), int64(3), object(4)
memory usage: 382.4+ KB


In [22]:
df23 = df23 [df23["type"] == 0]

In [23]:
df23 = df23[df23 ["confidence"]>= 80]

In [24]:
df23['acq_date'] = pd.to_datetime(df23['acq_date'])

In [25]:
df23['mois'] = df23['acq_date'].dt.to_period('M')

In [26]:
occurrence_par_mois = df23['mois'].value_counts().sort_index().reset_index()
occurrence_par_mois.columns = ['mois', 'Occurrences']

# Affichage des résultats
print(occurrence_par_mois)

       mois  Occurrences
0   2023-01            2
1   2023-02            6
2   2023-03            2
3   2023-04            2
4   2023-06            1
5   2023-07          736
6   2023-08         1180
7   2023-09           20
8   2023-10            6
9   2023-11           10
10  2023-12            1


In [27]:
df23.drop(["mois"], axis=1, inplace = True)

In [28]:
df23 = df23[df23['acq_date'].dt.month.isin([7, 8])]

In [29]:
df23.drop(["brightness","scan","track","satellite","instrument","confidence","version","bright_t31","frp","daynight","type"], axis =1 , inplace = True)

In [32]:
not_fire_data =  generate_not_fire_data(df23)

In [33]:
not_fire_data = not_fire_data [not_fire_data ['acq_date'].dt.month.isin([7, 8])]

In [34]:
not_fire_data["class"] = "not fire"
df23["class"] = "fire"

In [35]:
concatenated_df = pd.concat([df23, not_fire_data], ignore_index=True)
shuffled_df = concatenated_df.sample(frac=1).reset_index(drop=True)

In [36]:
shuffled_df.to_csv('data_Greece_.csv', index=False)